# Imports

In [4]:
import os
import torch
from diffusers import StableDiffusionPipeline
from transformers import CLIPTextModel, CLIPTokenizer
from PIL import Image, ImageOps, ImageFilter

c:\Users\asche\anaconda3\envs\ailab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Directories

In [ ]:
# Make sure the images directory exists
os.makedirs("./images", exist_ok=True)

In [ ]:
local_model_path = "stable_diffusion_model"

# Settings

In [5]:
# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
# Print the selected device
print("Using device:", device)

Using device: cpu


# Model

In [6]:
# from https://huggingface.co/CompVis/stable-diffusion-v1-4
model_id = "CompVis/stable-diffusion-v1-4"

In [7]:
# Function to check if the model is saved locally and load it
def load_model():
    if os.path.exists(local_model_path):
        print(f"Loading model from {local_model_path}...")
        pipe = StableDiffusionPipeline.from_pretrained(local_model_path, torch_dtype=torch.float32)
    else:
        print(f"Downloading model {model_id}...")
        pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
        # Save the initialized model for future use
        pipe.save_pretrained(local_model_path)
        print(f"Model saved to {local_model_path}")
    return pipe

Loading model from stable_diffusion_model...


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  7.78it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
# Load the model
pipe = load_model()

In [8]:
# Ensure that all necessary components are available
if not pipe.text_encoder:
    print("Text encoder model not found. Downloading...")
    pipe.text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder")
if not pipe.tokenizer:
    print("Tokenizer not found. Downloading...")
    pipe.tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")

# Optionally disable the safety checker
pipe.safety_checker = None

# Move the model to the selected device
pipe = pipe.to(device)

# Make images

In [9]:
# Post-process image based on style
def post_process_image(image, style="color"):
    if style == "black_and_white":
        image = ImageOps.grayscale(image)
    elif style == "outline":
        image = image.convert("L")  # Convert to grayscale
        image = image.filter(ImageFilter.FIND_EDGES)  # Find edges to create an outline effect
    return image

# Generate image based on prompt and apply style
def generate_image(prompt, filename, style="color", num_inference_steps=50):
    # Assuming pipe is a function that generates an image from a prompt
    image = pipe(prompt, num_inference_steps=num_inference_steps).images[0]
    image = post_process_image(image, style)
    image.save(filename)
    print(f"Image saved as {filename}")

In [ ]:
# Define the prompts and styles
prompts_and_styles = [
    ("viking children playing outside with a cat near a fire", "outline")
]

In [10]:
# Generate and save images
for i, (prompt, style) in enumerate(prompts_and_styles, start=1):
    filename = f"./images/generated_image_{i}_{style}.png"
    generate_image(prompt, filename, style)

100%|██████████| 50/50 [08:37<00:00, 10.35s/it]


Image saved as generated_image_1_outline.png
